# Data Collection and Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import files
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Emissions
Source: https://ourworldindata.org/co2-and-greenhouse-gas-emissions 

Citation: *Our World in Data*, by Hannah Ritchie, Max Roser, and Edouard Mathieu

In [2]:
raw_emissions = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataSci \
112/Final Project/annual-co2-emissions-per-country.csv")
raw_emissions

,Entity,Code,Year,Annual CO₂ emissions
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0
...,...,...,...,...
31344,Zimbabwe,ZWE,2017,9596071.0
31345,Zimbabwe,ZWE,2018,11795478.0
31346,Zimbabwe,ZWE,2019,11114607.0
31347,Zimbabwe,ZWE,2020,10607897.0


In [3]:
raw_emissions.describe()

,Year,Annual CO₂ emissions
count,31349.000000,3.134900e+04
mean,1936.980733,3.799881e+08
std,67.004912,1.799876e+09
min,1750.000000,0.000000e+00
25%,1897.000000,1.245760e+05
50%,1956.000000,3.109022e+06
75%,1990.000000,4.366016e+07
max,2021.000000,3.712385e+10


In [4]:
country_emissions = raw_emissions[~pd.isna(raw_emissions["Code"])]

In [5]:
year_emission_data = []
for year in range(1750, 2022):
  year_emission_data.append(
      country_emissions[country_emissions["Year"] == year]
      [["Entity", "Code", "Annual CO₂ emissions"]])

In [6]:
join_emissions = year_emission_data[0]
i = 1750
for year in year_emission_data[1:]:
  i += 1
  join_emissions = join_emissions.merge(
      year, on=("Entity", "Code"), how="outer", suffixes=("", "_" + str(i)))
join_emissions = join_emissions.fillna(0).rename(
    columns={"Annual CO₂ emissions":"Annual CO₂ emissions_1750"})
join_emissions

,Entity,Code,Annual CO₂ emissions_1750,Annual CO₂ emissions_1751,Annual CO₂ emissions_1752,Annual CO₂ emissions_1753,Annual CO₂ emissions_1754,Annual CO₂ emissions_1755,Annual CO₂ emissions_1756,Annual CO₂ emissions_1757,...,Annual CO₂ emissions_2012,Annual CO₂ emissions_2013,Annual CO₂ emissions_2014,Annual CO₂ emissions_2015,Annual CO₂ emissions_2016,Annual CO₂ emissions_2017,Annual CO₂ emissions_2018,Annual CO₂ emissions_2019,Annual CO₂ emissions_2020,Annual CO₂ emissions_2021
0,Andorra,AND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,487312.0,476320.0,461664.0,465328.0,468992.0,465328.0,494640.0,479984.0,448884.0,452888.0
1,Australia,AUS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,406150900.0,397887460.0,393952900.0,401793060.0,411264300.0,414358300.0,416283800.0,416356600.0,399922100.0,391187420.0
2,Brunei,BRN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9499821.0,7626062.0,8830987.0,6933294.0,7632904.0,9309383.0,9344423.0,10487509.0,10553166.0,10480519.0
3,Cambodia,KHM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5576016.0,5636730.0,6834254.0,8365610.0,10921712.0,12504842.0,14388181.0,18024642.0,18703284.0,19028598.0
4,Cote d'Ivoire,CIV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8540010.0,9679770.0,9903224.0,9554741.0,11982756.0,12001913.0,10352442.0,10535084.0,10963848.0,11706642.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Wallis and Futuna,WLF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25648.0,21984.0,21984.0,21984.0,25648.0,25648.0,25648.0,25648.0,27069.0,27818.0
219,Marshall Islands,MHL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,135568.0,139232.0,142896.0,142896.0,142896.0,146560.0,146560.0,146560.0,154682.0,158962.0
220,Micronesia (country),FSM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,124576.0,135568.0,135568.0,142896.0,142896.0,142896.0,142896.0,146560.0,154682.0,158962.0
221,Timor,TLS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,571584.0,531280.0,567920.0,549600.0,589904.0,633872.0,633872.0,685168.0,718671.0,736328.0


In [7]:
join_emissions.to_csv("Country Emissions by Year")
files.download("Country Emissions by Year")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Urban Population Percentage
Source: https://population.un.org/wup/DataQuery/

Citation: World Urbanization Prospects: The 2018 Revision by the United Nations

(Limits csv downloads to 35 years of data)

In [8]:
df_urban_pop_raw_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/\
DataSci 112/Final Project/Urban Pop 1.csv")
df_urban_pop_raw_1

,Annual Percentage of Population at Mid-Year Residing in Urban Areas,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,ISO 3166-1 numeric code,Location,Note,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,...,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0
1,900,World,NaN,29.6,30.0,30.4,30.8,31.2,31.6,32.1,...,37.7,38.0,38.2,38.5,38.9,39.3,39.8,40.1,40.5,40.8
2,901,More developed regions,a,54.8,55.4,56.0,56.6,57.3,57.9,58.6,...,68.8,69.2,69.5,69.8,70.0,70.3,70.6,70.8,71.0,71.2
3,902,Less developed regions,b,17.7,18.1,18.5,18.9,19.3,19.7,20.1,...,26.9,27.3,27.7,28.2,28.8,29.4,30.0,30.6,31.1,31.7
4,941,Least developed countries,c,7.5,7.7,7.9,8.1,8.3,8.5,8.7,...,14.5,15.0,15.5,16.0,16.6,17.1,17.6,18.0,18.4,18.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,882,Samoa,NaN,12.9,13.7,14.6,15.5,16.4,17.4,18.4,...,21.0,21.1,21.1,21.1,21.2,21.2,21.2,21.2,21.2,21.2
270,772,Tokelau,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
271,776,Tonga,NaN,12.9,13.4,13.9,14.4,14.9,15.5,16.1,...,20.3,20.4,20.5,20.7,20.9,21.2,21.4,21.6,21.8,22.0
272,798,Tuvalu,NaN,11.2,11.6,12.0,12.5,12.9,13.4,13.8,...,25.8,26.5,27.3,28.2,29.0,29.8,30.7,31.5,32.4,33.3


In [9]:
df_urban_pop_raw_1.columns = df_urban_pop_raw_1.iloc[0]
df_urban_pop_fixed_1 = df_urban_pop_raw_1.iloc[1:].drop("Note", axis='columns')
df_urban_pop_fixed_1

,ISO 3166-1 numeric code,Location,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,...,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0
1,900,World,29.6,30.0,30.4,30.8,31.2,31.6,32.1,32.5,...,37.7,38.0,38.2,38.5,38.9,39.3,39.8,40.1,40.5,40.8
2,901,More developed regions,54.8,55.4,56.0,56.6,57.3,57.9,58.6,59.2,...,68.8,69.2,69.5,69.8,70.0,70.3,70.6,70.8,71.0,71.2
3,902,Less developed regions,17.7,18.1,18.5,18.9,19.3,19.7,20.1,20.6,...,26.9,27.3,27.7,28.2,28.8,29.4,30.0,30.6,31.1,31.7
4,941,Least developed countries,7.5,7.7,7.9,8.1,8.3,8.5,8.7,9.0,...,14.5,15.0,15.5,16.0,16.6,17.1,17.6,18.0,18.4,18.8
5,934,"Less developed regions, excluding least ...",19.0,19.4,19.8,20.2,20.7,21.1,21.6,22.1,...,28.5,28.9,29.3,29.8,30.4,31.0,31.7,32.3,32.8,33.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,882,Samoa,12.9,13.7,14.6,15.5,16.4,17.4,18.4,18.7,...,21.0,21.1,21.1,21.1,21.2,21.2,21.2,21.2,21.2,21.2
270,772,Tokelau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
271,776,Tonga,12.9,13.4,13.9,14.4,14.9,15.5,16.1,16.5,...,20.3,20.4,20.5,20.7,20.9,21.2,21.4,21.6,21.8,22.0
272,798,Tuvalu,11.2,11.6,12.0,12.5,12.9,13.4,13.8,14.3,...,25.8,26.5,27.3,28.2,29.0,29.8,30.7,31.5,32.4,33.3


In [10]:
df_urban_pop_raw_2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/\
DataSci 112/Final Project/Urban Pop 2.csv")
df_urban_pop_raw_2.columns = df_urban_pop_raw_2.iloc[0]
df_urban_pop_fixed_2 = df_urban_pop_raw_2.iloc[1:].drop("Note", axis='columns')
df_urban_pop_fixed_2

,ISO 3166-1 numeric code,Location,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
1,900,World,41.2,41.6,41.9,42.3,42.6,43.0,43.3,43.7,...,51.7,52.1,52.6,53.0,53.5,53.9,54.4,54.8,55.3,55.7
2,901,More developed regions,71.4,71.6,71.8,72.0,72.2,72.4,72.5,72.7,...,77.2,77.4,77.6,77.8,78.0,78.1,78.3,78.5,78.7,78.9
3,902,Less developed regions,32.2,32.8,33.3,33.8,34.4,34.9,35.4,35.9,...,46.1,46.7,47.3,47.9,48.4,49.0,49.5,50.1,50.6,51.2
4,941,Least developed countries,19.3,19.7,20.2,20.6,21.1,21.5,21.9,22.3,...,29.5,30.0,30.5,31.0,31.5,32.0,32.5,33.0,33.6,34.1
5,934,"Less developed regions, excluding least ...",34.0,34.6,35.1,35.7,36.2,36.8,37.3,37.9,...,49.0,49.7,50.3,50.9,51.5,52.1,52.7,53.3,53.9,54.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,882,Samoa,21.2,21.2,21.2,21.2,21.2,21.2,21.2,21.3,...,20.1,19.9,19.6,19.4,19.1,18.9,18.7,18.5,18.2,18.1
270,772,Tokelau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
271,776,Tonga,22.3,22.5,22.6,22.6,22.7,22.7,22.7,22.8,...,23.4,23.4,23.4,23.4,23.3,23.3,23.2,23.2,23.1,23.1
272,798,Tuvalu,34.3,35.5,36.8,38.0,39.3,40.7,42.0,42.8,...,54.8,55.8,56.8,57.8,58.8,59.7,60.6,61.5,62.4,63.2


In [11]:
df_urban_pop_raw_3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/\
DataSci 112/Final Project/Urban Pop 3.csv")
df_urban_pop_raw_3.columns = df_urban_pop_raw_3.iloc[0]
df_urban_pop_fixed_3 = df_urban_pop_raw_3.iloc[1:].drop("Note", axis='columns')
df_urban_pop_fixed_3

,ISO 3166-1 numeric code,Location,2020.0,2021.0,2022.0
1,900,World,56.2,56.6,57.0
2,901,More developed regions,79.1,79.3,79.5
3,902,Less developed regions,51.7,52.2,52.8
4,941,Least developed countries,34.6,35.2,35.7
5,934,"Less developed regions, excluding least ...",55.1,55.6,56.2
...,...,...,...,...,...
269,882,Samoa,17.9,17.7,17.6
270,772,Tokelau,0.0,0.0,0.0
271,776,Tonga,23.1,23.1,23.1
272,798,Tuvalu,64.0,64.8,65.5


In [12]:
urban_pop = df_urban_pop_fixed_1.merge(
    df_urban_pop_fixed_2, on=("Location", "ISO 3166-1 numeric code")).merge(
        df_urban_pop_fixed_3, on=("Location", "ISO 3166-1 numeric code"))
columns = pd.Series(urban_pop.columns)
columns[2:] = columns[2:].astype(int)
urban_pop.columns = columns
urban_pop

,ISO 3166-1 numeric code,Location,1950,1951,1952,1953,1954,1955,1956,1957,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,900,World,29.6,30.0,30.4,30.8,31.2,31.6,32.1,32.5,...,53.0,53.5,53.9,54.4,54.8,55.3,55.7,56.2,56.6,57.0
1,901,More developed regions,54.8,55.4,56.0,56.6,57.3,57.9,58.6,59.2,...,77.8,78.0,78.1,78.3,78.5,78.7,78.9,79.1,79.3,79.5
2,902,Less developed regions,17.7,18.1,18.5,18.9,19.3,19.7,20.1,20.6,...,47.9,48.4,49.0,49.5,50.1,50.6,51.2,51.7,52.2,52.8
3,941,Least developed countries,7.5,7.7,7.9,8.1,8.3,8.5,8.7,9.0,...,31.0,31.5,32.0,32.5,33.0,33.6,34.1,34.6,35.2,35.7
4,934,"Less developed regions, excluding least ...",19.0,19.4,19.8,20.2,20.7,21.1,21.6,22.1,...,50.9,51.5,52.1,52.7,53.3,53.9,54.5,55.1,55.6,56.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,882,Samoa,12.9,13.7,14.6,15.5,16.4,17.4,18.4,18.7,...,19.4,19.1,18.9,18.7,18.5,18.2,18.1,17.9,17.7,17.6
269,772,Tokelau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,776,Tonga,12.9,13.4,13.9,14.4,14.9,15.5,16.1,16.5,...,23.4,23.3,23.3,23.2,23.2,23.1,23.1,23.1,23.1,23.1
271,798,Tuvalu,11.2,11.6,12.0,12.5,12.9,13.4,13.8,14.3,...,57.8,58.8,59.7,60.6,61.5,62.4,63.2,64.0,64.8,65.5


In [13]:
urban_pop.to_csv("Country Urban Population Percentage by Year")
files.download("Country Urban Population Percentage by Year")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Population
Source: https://population.un.org/wup/DataQuery/

Citation: World Urbanization Prospects: The 2018 Revision by the United Nations

(Limits csv downloads to 35 years of data)

In [14]:
df_pop_raw_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/\
DataSci 112/Final Project/Pop 1.csv")
df_pop_raw_1

,Annual Total Population at Mid-Year (thousands),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,ISO 3166-1 numeric code,Location,Note,1950,1951,1952,1953,1954,1955,1956,...,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984
1,900,World,NaN,2 536 275,2 583 817,2 630 584,2 677 230,2 724 302,2 772 243,2 821 383,...,4 079 087,4 154 288,4 229 201,4 304 377,4 380 586,4 458 412,4 537 846,4 618 776,4 701 531,4 786 484
2,901,More developed regions,a,814 865,824 213,834 074,844 264,854 632,865 069,875 507,...,1 049 414,1 056 821,1 063 964,1 070 879,1 077 624,1 084 244,1 090 731,1 097 084,1 103 358,1 109 627
3,902,Less developed regions,b,1 721 410,1 759 604,1 796 510,1 832 967,1 869 671,1 907 173,1 945 876,...,3 029 674,3 097 467,3 165 237,3 233 498,3 302 962,3 374 167,3 447 114,3 521 693,3 598 173,3 676 857
4,941,Least developed countries,c,195 259,199 052,202 905,206 885,211 045,215 421,220 033,...,347 093,355 654,364 581,373 854,383 430,393 279,403 409,413 847,424 614,435 738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,882,Samoa,NaN,82,84,87,89,92,94,97,...,151,152,153,154,155,156,156,157,158,159
270,772,Tokelau,NaN,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
271,776,Tonga,NaN,47,49,51,53,54,55,56,...,88,89,90,91,92,93,93,94,94,94
272,798,Tuvalu,NaN,5,5,5,5,5,6,6,...,8,8,8,8,8,8,8,8,8,9


In [15]:
df_pop_raw_1.columns = df_pop_raw_1.iloc[0]
df_pop_fixed_1 = df_pop_raw_1.iloc[1:].drop("Note", axis='columns')
int_list_1 = list(df_pop_fixed_1.columns)
int_list_1.remove("Location")
for col in int_list_1:
  df_pop_fixed_1[col] = df_pop_fixed_1[col].astype(str).str.replace(
      " ", "").astype(int)
df_pop_fixed_1

,ISO 3166-1 numeric code,Location,1950,1951,1952,1953,1954,1955,1956,1957,...,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984
1,900,World,2536275,2583817,2630584,2677230,2724302,2772243,2821383,2871952,...,4079087,4154288,4229201,4304377,4380586,4458412,4537846,4618776,4701531,4786484
2,901,More developed regions,814865,824213,834074,844264,854632,865069,875507,885914,...,1049414,1056821,1063964,1070879,1077624,1084244,1090731,1097084,1103358,1109627
3,902,Less developed regions,1721410,1759604,1796510,1832967,1869671,1907173,1945876,1986038,...,3029674,3097467,3165237,3233498,3302962,3374167,3447114,3521693,3598173,3676857
4,941,Least developed countries,195259,199052,202905,206885,211045,215421,220033,224885,...,347093,355654,364581,373854,383430,393279,403409,413847,424614,435738
5,934,"Less developed regions, excluding least ...",1526151,1560552,1593605,1626082,1658626,1691752,1725843,1761153,...,2682580,2741813,2800656,2859643,2919532,2980888,3043706,3107846,3173559,3241119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,882,Samoa,82,84,87,89,92,94,97,100,...,151,152,153,154,155,156,156,157,158,159
270,772,Tokelau,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
271,776,Tonga,47,49,51,53,54,55,56,57,...,88,89,90,91,92,93,93,94,94,94
272,798,Tuvalu,5,5,5,5,5,6,6,6,...,8,8,8,8,8,8,8,8,8,9


In [16]:
df_pop_raw_2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/\
DataSci 112/Final Project/Pop 2.csv")
df_pop_raw_2
df_pop_raw_2.columns = df_pop_raw_2.iloc[0]
df_pop_fixed_2 = df_pop_raw_2.iloc[1:].drop("Note", axis='columns')
int_list_2 = list(df_pop_fixed_2.columns)
int_list_2.remove("Location")
for col in int_list_2:
  df_pop_fixed_2[col] = df_pop_fixed_2[col].astype(str).str.replace(
      " ", "").astype(int)
df_pop_fixed_2

,ISO 3166-1 numeric code,Location,1985,1986,1987,1988,1989,1990,1991,1992,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,900,World,4873782,4963633,5055636,5148557,5240735,5330943,5418759,5504401,...,6958169,7043009,7128177,7213426,7298453,7383009,7466964,7550262,7632819,7714577
2,901,More developed regions,1115935,1122313,1128727,1135070,1141197,1146999,1152455,1157593,...,1235143,1239298,1243055,1246520,1249864,1253207,1256576,1259922,1263200,1266335
3,902,Less developed regions,3757847,3841320,3926909,4013487,4099538,4183944,4266304,4346808,...,5723027,5803711,5885122,5966906,6048590,6129802,6210388,6290340,6369620,6448242
4,941,Least developed countries,447241,459116,471365,484033,497177,510828,525020,539720,...,848792,869298,890423,912094,934192,956631,979388,1002486,1025937,1049765
5,934,"Less developed regions, excluding least ...",3310606,3382204,3455544,3529454,3602362,3673117,3741284,3807088,...,4874235,4934413,4994699,5054812,5114397,5173171,5231000,5287854,5343683,5398477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,882,Samoa,160,161,161,161,162,163,164,166,...,186,188,189,191,192,194,195,196,198,199
270,772,Tokelau,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
271,776,Tonga,94,94,94,95,95,95,95,95,...,104,105,105,105,106,106,107,108,109,110
272,798,Tuvalu,9,9,9,9,9,9,9,9,...,11,11,11,11,11,11,11,11,11,11


In [17]:
df_pop_raw_3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/\
DataSci 112/Final Project/Pop 3.csv")
df_pop_raw_3
df_pop_raw_3.columns = df_pop_raw_3.iloc[0]
df_pop_fixed_3 = df_pop_raw_3.iloc[1:].drop("Note", axis='columns')
int_list_3 = list(df_pop_fixed_3.columns)
int_list_3.remove("Location")
for col in int_list_3:
  df_pop_fixed_3[col] = df_pop_fixed_3[col].astype(str).str.replace(
      " ", "").astype(int)
df_pop_fixed_3

,ISO 3166-1 numeric code,Location,2020,2021,2022
1,900,World,7795482,7875465,7954469
2,901,More developed regions,1269277,1272013,1274563
3,902,Less developed regions,6526205,6603452,6679906
4,941,Least developed countries,1073984,1098591,1123563
5,934,"Less developed regions, excluding least ...",5452221,5504861,5556343
...,...,...,...,...,...
269,882,Samoa,200,201,203
270,772,Tokelau,1,1,1
271,776,Tonga,111,112,113
272,798,Tuvalu,11,12,12


In [18]:
pop = df_pop_fixed_1.merge(
    df_pop_fixed_2, on=("Location", "ISO 3166-1 numeric code")).merge(
        df_pop_fixed_3, on=("Location", "ISO 3166-1 numeric code"))
pop

,ISO 3166-1 numeric code,Location,1950,1951,1952,1953,1954,1955,1956,1957,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,900,World,2536275,2583817,2630584,2677230,2724302,2772243,2821383,2871952,...,7213426,7298453,7383009,7466964,7550262,7632819,7714577,7795482,7875465,7954469
1,901,More developed regions,814865,824213,834074,844264,854632,865069,875507,885914,...,1246520,1249864,1253207,1256576,1259922,1263200,1266335,1269277,1272013,1274563
2,902,Less developed regions,1721410,1759604,1796510,1832967,1869671,1907173,1945876,1986038,...,5966906,6048590,6129802,6210388,6290340,6369620,6448242,6526205,6603452,6679906
3,941,Least developed countries,195259,199052,202905,206885,211045,215421,220033,224885,...,912094,934192,956631,979388,1002486,1025937,1049765,1073984,1098591,1123563
4,934,"Less developed regions, excluding least ...",1526151,1560552,1593605,1626082,1658626,1691752,1725843,1761153,...,5054812,5114397,5173171,5231000,5287854,5343683,5398477,5452221,5504861,5556343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,882,Samoa,82,84,87,89,92,94,97,100,...,191,192,194,195,196,198,199,200,201,203
269,772,Tokelau,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
270,776,Tonga,47,49,51,53,54,55,56,57,...,105,106,106,107,108,109,110,111,112,113
271,798,Tuvalu,5,5,5,5,5,6,6,6,...,11,11,11,11,11,11,11,11,12,12


In [19]:
pop.to_csv("Country Population by Year")
files.download("Country Population by Year")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## GDP per Capita
Source: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD 

Citation: World Bank national accounts data, and OECD National Accounts data files

In [20]:
gdp_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataSci 112/Final Project\
/GDP.csv")
gdp_raw

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Last Updated Date,2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country Name,Country Code,Indicator Name,Indicator Code,1960.000000,1961.000000,1962.000000,1963.000000,1964.000000,1965.000000,...,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2020.000000,2021.000000,NaN
4,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,26515.678080,26942.307976,28421.386493,28451.273745,29326.708058,30220.594523,31650.760537,24487.863560,29342.100858,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Kosovo,XKX,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3704.784221,3902.676013,3520.766449,3759.560246,4009.380987,4384.048892,4416.108358,4310.811183,5269.783901,NaN
266,"Yemen, Rep.",YEM,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1497.747941,1557.601406,1488.416482,1069.817122,893.716573,701.714878,NaN,NaN,NaN,NaN
267,South Africa,ZAF,GDP per capita (current US$),NY.GDP.PCAP.CD,529.561923,543.042224,560.699395,601.599951,642.688431,681.131112,...,7441.230854,6965.137897,6204.929901,5735.066787,6734.475153,7048.522211,6688.787271,5741.643129,7055.044776,NaN
268,Zambia,ZMB,GDP per capita (current US$),NY.GDP.PCAP.CD,228.567399,216.274674,208.562685,209.453362,236.941713,296.022427,...,1840.320553,1724.576220,1307.909649,1249.923143,1495.752138,1475.204538,1268.120941,956.831364,1137.343633,NaN


In [21]:
cols = pd.Series(gdp_raw.iloc[3]).fillna("x")
cols[4:-1] = cols[4:-1].astype(int)
gdp_raw.columns = cols
gdp_fixed = gdp_raw.iloc[4:].drop(["Indicator Name", "Indicator Code", "x"], axis="columns")
gdp_fixed

3,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
4,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25609.955724,26515.678080,26942.307976,28421.386493,28451.273745,29326.708058,30220.594523,31650.760537,24487.863560,29342.100858
5,Africa Eastern and Southern,AFE,162.913034,162.551683,172.002460,199.189238,179.387799,198.230368,209.414665,211.707060,...,1759.182395,1730.394686,1719.183721,1538.552268,1443.692371,1628.586788,1564.734340,1512.270553,1363.540741,1549.772730
6,Afghanistan,AFG,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,...,663.141053,651.987862,628.146804,592.476537,520.252064,530.149831,502.056771,500.522664,516.866552,368.754614
7,Africa Western and Central,AFW,106.976475,112.047561,117.730633,122.278715,130.599963,137.186142,142.895375,127.303606,...,1953.407033,2149.295219,2243.271464,1876.623483,1645.023767,1585.911930,1731.311792,1749.303317,1683.436391,1757.030626
8,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4962.552072,5101.983876,5059.080441,3100.830685,1709.515534,2283.214233,2487.500996,2142.238757,1603.993477,1953.533757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3410.859780,3704.784221,3902.676013,3520.766449,3759.560246,4009.380987,4384.048892,4416.108358,4310.811183,5269.783901
266,"Yemen, Rep.",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1349.990295,1497.747941,1557.601406,1488.416482,1069.817122,893.716573,701.714878,NaN,NaN,NaN
267,South Africa,ZAF,529.561923,543.042224,560.699395,601.599951,642.688431,681.131112,718.118179,775.152812,...,8173.869138,7441.230854,6965.137897,6204.929901,5735.066787,6734.475153,7048.522211,6688.787271,5741.643129,7055.044776
268,Zambia,ZMB,228.567399,216.274674,208.562685,209.453362,236.941713,296.022427,334.672528,350.653425,...,1729.647471,1840.320553,1724.576220,1307.909649,1249.923143,1495.752138,1475.204538,1268.120941,956.831364,1137.343633


In [22]:
gdp_fixed.to_csv("Country GDP per Capita by Year")
files.download("Country GDP per Capita by Year")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Energy Sectors

Source: https://www.kaggle.com/datasets/pralabhpoudel/world-energy-consumption

Citation: *Our World in Data*, by Hannah Ritchie, Max Roser, and Edouard Mathieu

In [23]:
df_energy_sector_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataSci\
 112/Final Project/World Energy Consumption.csv")
df_energy_sector_raw

,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17427,ZWE,Zimbabwe,2015,-25.013,-10.847,NaN,NaN,NaN,NaN,-0.789,...,0.579,NaN,2.503057e+10,0.0,NaN,NaN,NaN,NaN,0.0,NaN
17428,ZWE,Zimbabwe,2016,-37.694,-12.257,NaN,NaN,NaN,NaN,-14.633,...,0.641,NaN,2.515176e+10,0.0,NaN,NaN,NaN,NaN,0.0,NaN
17429,ZWE,Zimbabwe,2017,8.375,1.697,NaN,NaN,NaN,NaN,NaN,...,0.773,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
17430,ZWE,Zimbabwe,2018,22.555,4.952,NaN,NaN,NaN,NaN,NaN,...,0.970,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN


In [24]:
columns = ["biofuel_share_energy", "coal_share_energy", "oil_share_energy", 
           "low_carbon_share_energy", "fossil_share_energy", "gas_share_energy",
           ]
for column in columns:
  df = df_energy_sector_raw[["iso_code", "country", "year", column]]
  year_data = []
  for year in range(df["year"].min(), df["year"].max() + 1):
    year_data.append(df[df["year"] == year][["iso_code", "country", column]])
  join_data = year_data[0]
  i = df["year"].min()
  for year in year_data[1:]:
    i += 1
    join_data = join_data.merge(
        year, on=("country", "iso_code"), how="outer", 
        suffixes=("", "_" + str(i)))
  join_data = join_data.rename(
      columns={column:column + "_" + str(df["year"].min())})
  display(join_data)
  join_data.to_csv("Country " + column + " by Year")
  files.download("Country " + column + " by Year")

,iso_code,country,biofuel_share_energy_1900,biofuel_share_energy_1901,biofuel_share_energy_1902,biofuel_share_energy_1903,biofuel_share_energy_1904,biofuel_share_energy_1905,biofuel_share_energy_1906,biofuel_share_energy_1907,...,biofuel_share_energy_2011,biofuel_share_energy_2012,biofuel_share_energy_2013,biofuel_share_energy_2014,biofuel_share_energy_2015,biofuel_share_energy_2016,biofuel_share_energy_2017,biofuel_share_energy_2018,biofuel_share_energy_2019,biofuel_share_energy_2020
0,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001,0.001,0.001,0.004,0.007,0.009,0.012,0.015,0.018,NaN
2,ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DZA,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
4,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,PSE,Palestine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,SSD,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,OWID_KOS,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,MNE,Montenegro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iso_code,country,coal_share_energy_1900,coal_share_energy_1901,coal_share_energy_1902,coal_share_energy_1903,coal_share_energy_1904,coal_share_energy_1905,coal_share_energy_1906,coal_share_energy_1907,...,coal_share_energy_2011,coal_share_energy_2012,coal_share_energy_2013,coal_share_energy_2014,coal_share_energy_2015,coal_share_energy_2016,coal_share_energy_2017,coal_share_energy_2018,coal_share_energy_2019,coal_share_energy_2020
0,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.547,24.069,23.750,24.151,22.485,23.254,22.653,22.716,22.507,NaN
2,ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DZA,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.689,0.669,0.342,0.356,0.250,0.092,0.339,0.865,0.825,NaN
4,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,PSE,Palestine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,SSD,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,OWID_KOS,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,MNE,Montenegro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iso_code,country,gas_share_energy_1900,gas_share_energy_1901,gas_share_energy_1902,gas_share_energy_1903,gas_share_energy_1904,gas_share_energy_1905,gas_share_energy_1906,gas_share_energy_1907,...,gas_share_energy_2011,gas_share_energy_2012,gas_share_energy_2013,gas_share_energy_2014,gas_share_energy_2015,gas_share_energy_2016,gas_share_energy_2017,gas_share_energy_2018,gas_share_energy_2019,gas_share_energy_2020
0,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.927,24.823,24.499,24.446,25.823,25.861,26.836,27.624,27.199,NaN
2,ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DZA,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,57.700,58.853,59.822,61.635,61.415,62.573,63.291,64.590,64.163,NaN
4,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,PSE,Palestine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,SSD,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,OWID_KOS,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,MNE,Montenegro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Electrification

Source: https://data.worldbank.org/indicator/EG.ELC.ACCS.ZS 

Citation: World Bank Global Electrification Database from "Tracking SDG 7: The Energy Progress Report" led jointly by the custodian agencies: the International Energy Agency ( IEA ), the International Renewable Energy Agency ( IRENA ), the United Nations Statistics Division ( UNSD ), the World Bank and the World Health Organization ( WHO ).

In [25]:
df_elec_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataSci 112/Final Project/Electrification Data.csv")
df_elec_fixed = df_elec_raw.iloc[4:]
cols = df_elec_raw.iloc[3]
cols.iloc[4:-1] = cols.iloc[4:-1].astype(int)
df_elec_fixed.columns = cols
df_elec_fixed = df_elec_fixed.iloc[:, :-2].drop(
    ["Indicator Code", "Indicator Name"], axis="columns")
df_elec_fixed

<ipython-input-25-6b3559ead671>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cols.iloc[4:-1] = cols.iloc[4:-1].astype(int)


3,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
5,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.948628,31.682318,31.610692,31.824950,33.744405,38.733352,40.092163,42.880977,44.073912,45.609604
6,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.222019,69.099998,68.290649,89.500000,71.500000,97.699997,97.699997,96.616135,97.699997,97.699997
7,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45.896347,44.137410,47.029335,47.587145,46.750946,50.931399,48.835062,51.238518,51.260220,52.082053
8,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.599998,37.131321,38.278030,32.000000,42.000000,41.813129,43.013260,45.290001,45.642799,46.890610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,99.750000,99.750000,99.680000,99.860001,99.800003,99.800003,NaN,100.000000,NaN
266,"Yemen, Rep.",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.864506,52.410000,75.599998,66.099998,67.310928,68.859505,79.199997,62.000000,72.751076,73.757927
267,South Africa,ZAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83.599998,85.300003,85.199997,85.900002,85.300003,83.900002,84.400002,84.699997,85.000000,84.385536
268,Zambia,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.159010,29.012726,29.886272,27.900000,31.100000,35.425453,40.299999,40.317890,43.000000,44.524475


In [26]:
for i in range(2, 100):
  if df_elec_fixed.iloc[:, i].min() > 0:
    print(i)
    break

32


In [27]:
for col in df_elec_fixed.columns[2:32]:
  df_elec_fixed = df_elec_fixed.drop(col, axis="columns")
df_elec_fixed

3,Country Name,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4,Aruba,ABW,100.0,99.153656,99.197128,99.239914,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
5,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.948628,31.682318,31.610692,31.824950,33.744405,38.733352,40.092163,42.880977,44.073912,45.609604
6,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.222019,69.099998,68.290649,89.500000,71.500000,97.699997,97.699997,96.616135,97.699997,97.699997
7,Africa Western and Central,AFW,NaN,NaN,NaN,31.575130,32.609692,33.605668,32.629322,32.461797,...,45.896347,44.137410,47.029335,47.587145,46.750946,50.931399,48.835062,51.238518,51.260220,52.082053
8,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.599998,37.131321,38.278030,32.000000,42.000000,41.813129,43.013260,45.290001,45.642799,46.890610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Kosovo,XKX,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,NaN,99.750000,99.750000,99.680000,99.860001,99.800003,99.800003,NaN,100.000000,NaN
266,"Yemen, Rep.",YEM,NaN,NaN,44.100000,40.774715,41.985134,43.193512,44.399170,42.600000,...,61.864506,52.410000,75.599998,66.099998,67.310928,68.859505,79.199997,62.000000,72.751076,73.757927
267,South Africa,ZAF,NaN,NaN,NaN,NaN,NaN,NaN,57.600000,69.483353,...,83.599998,85.300003,85.199997,85.900002,85.300003,83.900002,84.400002,84.699997,85.000000,84.385536
268,Zambia,ZMB,13.9,12.752703,19.200000,14.213207,14.941759,15.668272,17.300000,17.112459,...,28.159010,29.012726,29.886272,27.900000,31.100000,35.425453,40.299999,40.317890,43.000000,44.524475


In [28]:
df_elec_fixed.to_csv("Country Electrification by Year")
files.download("Country Electrification by Year")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Foreign Direct Investment

Source: https://data.worldbank.org/indicator/BX.KLT.DINV.CD.WD

Citation: International Monetary Fund, Balance of Payments database, supplemented by data from the United Nations Conference on Trade and Development and official national sources.








In [29]:
df_fdi_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataSci 112\
/Final Project/FDI.csv")
df_fdi_raw

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Last Updated Date,2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2.013000e+03,2.014000e+03,2.015000e+03,2.016000e+03,2.017000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.021000e+03,NaN
4,Aruba,ABW,"Foreign direct investment, net inflows (BoP, c...",BX.KLT.DINV.CD.WD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.263714e+08,2.506181e+08,-2.877586e+07,2.755127e+07,1.624636e+08,1.359212e+08,-7.522654e+07,1.610670e+08,1.335451e+08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Kosovo,XKX,"Foreign direct investment, net inflows (BoP, c...",BX.KLT.DINV.CD.WD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.714922e+08,1.998884e+08,3.433506e+08,2.437871e+08,2.868352e+08,3.185430e+08,2.850816e+08,3.946253e+08,5.009279e+08,NaN
266,"Yemen, Rep.",YEM,"Foreign direct investment, net inflows (BoP, c...",BX.KLT.DINV.CD.WD,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.335709e+08,-2.331047e+08,-1.544481e+07,-5.610000e+08,-2.698500e+08,-2.820983e+08,-3.709828e+08,NaN,NaN,NaN
267,South Africa,ZAF,"Foreign direct investment, net inflows (BoP, c...",BX.KLT.DINV.CD.WD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.232519e+09,5.791659e+09,1.521140e+09,2.215307e+09,2.058580e+09,5.569462e+09,5.116098e+09,3.153553e+09,4.129614e+10,NaN
268,Zambia,ZMB,"Foreign direct investment, net inflows (BoP, c...",BX.KLT.DINV.CD.WD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.099800e+09,1.507800e+09,1.582667e+09,6.628139e+08,1.107520e+09,4.084385e+08,5.479679e+08,2.452055e+08,-8.230827e+08,NaN


In [ ]:
columns = df_fdi_raw.iloc[3]
columns[4:-1] = columns[4:-1].astype(int)
df_fdi_raw.columns = columns
df_fdi_raw = df_fdi_raw.iloc[4:, :-1].drop(["Indicator Name", "Indicator Code"],
                                           axis=1)

In [31]:
for i in range(2, 20):
  if df_fdi_raw.iloc[:, i].sum() != 0:
    print(i)
    break

12


In [32]:
df_fdi_fixed = df_fdi_raw.drop(df_fdi_raw.columns[2:12], axis=1)
df_fdi_fixed

3,Country Name,Country Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
4,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.146927e+08,2.263714e+08,2.506181e+08,-2.877586e+07,2.755127e+07,1.624636e+08,1.359212e+08,-7.522654e+07,1.610670e+08,1.335451e+08
5,Africa Eastern and Southern,AFE,4.217099e+08,3.264909e+08,2.188882e+08,1.765591e+08,8.985156e+08,3.168347e+08,4.112611e+08,6.710362e+07,...,2.463634e+10,2.153425e+10,2.768142e+10,2.877423e+10,1.666027e+10,1.010829e+10,1.358898e+10,1.499645e+10,1.289063e+10,5.297461e+10
6,Afghanistan,AFG,2.300000e+05,4.500000e+05,1.500000e+05,2.700000e+05,NaN,NaN,4.000000e+04,-2.400000e+05,...,5.682366e+07,4.831135e+07,4.297526e+07,1.691466e+08,9.359132e+07,5.153390e+07,1.194351e+08,2.340455e+07,1.297015e+07,2.060098e+07
7,Africa Western and Central,AFW,4.074600e+08,3.886800e+08,4.766600e+08,6.424100e+08,5.182691e+08,9.691052e+08,6.053788e+08,7.787905e+08,...,2.091269e+10,1.915755e+10,1.659803e+10,1.564317e+10,1.393704e+10,1.766242e+10,1.549284e+10,1.245296e+10,1.090646e+10,2.067716e+10
8,Angola,AGO,2.400000e+06,1.860000e+06,2.160000e+06,7.540000e+06,6.690000e+06,5.000000e+04,-1.170000e+06,NaN,...,-1.464628e+09,-7.120017e+09,3.657515e+09,1.002822e+10,-1.795176e+08,-7.397295e+09,-6.456076e+09,-4.098479e+09,-1.866468e+09,-4.355117e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.930898e+08,3.714922e+08,1.998884e+08,3.433506e+08,2.437871e+08,2.868352e+08,3.185430e+08,2.850816e+08,3.946253e+08,5.009279e+08
266,"Yemen, Rep.",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.423919e+07,-1.335709e+08,-2.331047e+08,-1.544481e+07,-5.610000e+08,-2.698500e+08,-2.820983e+08,-3.709828e+08,NaN,NaN
267,South Africa,ZAF,3.345999e+08,2.600609e+08,1.144782e+08,2.737914e+07,6.961236e+08,1.879628e+08,1.840000e+07,-1.219000e+08,...,4.626029e+09,8.232519e+09,5.791659e+09,1.521140e+09,2.215307e+09,2.058580e+09,5.569462e+09,5.116098e+09,3.153553e+09,4.129614e+10
268,Zambia,ZMB,9.520000e+06,8.420000e+06,2.900000e+07,3.200000e+07,3.800000e+07,3.800000e+07,3.100000e+07,4.296000e+07,...,1.731500e+09,2.099800e+09,1.507800e+09,1.582667e+09,6.628139e+08,1.107520e+09,4.084385e+08,5.479679e+08,2.452055e+08,-8.230827e+08


In [33]:
df_fdi_fixed.to_csv("Country FDI by Year.csv")
files.download("Country FDI by Year.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Fortune Global 500

Source: [fortune.com/ranking/global500](https://)

Citation: Fortune 500 Ranking List

In [ ]:
global500_collection = {}
for year in range(1995, 2023):
  response = requests.get(
    "https://fortune.com/ranking/global500/" + str(year) + "/search/")
  soup = BeautifulSoup(response.text, "html.parser")
  text = str(soup.find("body").find("script"))
  soup_json = text.replace(
      '<script id="__NEXT_DATA__" type="application/json">', '').replace(
          '</script>', '')
  site_json=json.loads(soup_json)
  raw = pd.json_normalize(site_json, ["props", "pageProps", 
                                            "franchiseList", "items"])
  
  if year < 2015: 
    raw = raw[["name",  "data.Headquarters Country", "data.Revenues"]]
    fixed = raw.rename(columns = {
    "name" : "Company", "data.Headquarters Country" : "Country",
     "data.Revenues" : "Revenue", "data.Headquarters City":"City"})
    fixed["Revenue"] = fixed["Revenue"].str.replace(",", "").astype(int)
  
  elif year < 2016: 
    raw = raw[["name", "data.Country", "data.Headquarters City", 
               "data.Revenues"]]
    fixed = raw.rename(columns = {
    "name" : "Company", "data.Country" : "Country",
     "data.Revenues" : "Revenue", "data.Headquarters City":"City"})
    fixed["Revenue"] = fixed["Revenue"].str.replace(",", "").astype(int)  

  elif year < 2019:
    raw = raw[["name", "data.Country", "data.Headquarters City", 
               "data.Revenues ($M)"]]
    fixed = raw.rename(columns = {
    "name" : "Company", "data.Country" : "Country",
     "data.Revenues ($M)" : "Revenue", "data.Headquarters City":"City"})
    fixed["Revenue"] = fixed["Revenue"].str.replace(",", "").str.replace(
        "$", "").astype(int) * 1000000

  elif year < 2023:
    raw = raw[["name", "data.Country / Territory", "data.Headquarters City",
               "data.Revenues ($M)"]]
    fixed = raw.rename(columns = {
    "name" : "Company", "data.Country / Territory" : "Country", 
    "data.Revenues ($M)" : "Revenue", "data.Headquarters City":"City"})
    fixed["Revenue"] = fixed["Revenue"].str.replace(",", "").str.replace(
        "$", "").astype(float).round().astype(int) * 1000000 

  fixed["Year"] = year      
  global500_collection[year] = fixed

In [35]:
df_fortune = global500_collection[1995]
for year in range(1996, 2023):
  df_fortune = pd.concat([df_fortune, global500_collection[year]], axis=0)
df_fortune.index = [i for i in range(13686)]
df_fortune

,Company,Country,Revenue,Year,City
0,Mitsubishi Corporation,Japan,17583560,1995,NaN
1,"Mitsui & Co., Ltd.",Japan,17149050,1995,NaN
2,Itochu Corporation,Japan,16782470,1995,NaN
3,Sumitomo Corporation,Japan,16247590,1995,NaN
4,General Motors Corporation,U.S.,15495120,1995,NaN
...,...,...,...,...,...
13681,DSV,Denmark,28988000000,2022,Hedehusene
13682,ABB,Switzerland,28945000000,2022,Zurich
13683,Mondelēz International,U.S.,28720000000,2022,Chicago
13684,Danone,France,28708000000,2022,Paris


In [36]:
df_fortune["Country"] = df_fortune["Country"].replace(
    to_replace=['U.S.', 'Britain', 'Britain/Neth.', 'Netherlands Antilles', 
                'U.A.E.', 'U.A.E', "Britain/Netherlands", "Belgium/Netherlands"],
    value=['United States of America', 'United Kingdom', 'United Kingdom', 
           'Netherlands', 'United Arab Emirates', 'United Arab Emirates', 
           'United Kingdom', "Belgium"]
)
df_fortune

,Company,Country,Revenue,Year,City
0,Mitsubishi Corporation,Japan,17583560,1995,NaN
1,"Mitsui & Co., Ltd.",Japan,17149050,1995,NaN
2,Itochu Corporation,Japan,16782470,1995,NaN
3,Sumitomo Corporation,Japan,16247590,1995,NaN
4,General Motors Corporation,United States of America,15495120,1995,NaN
...,...,...,...,...,...
13681,DSV,Denmark,28988000000,2022,Hedehusene
13682,ABB,Switzerland,28945000000,2022,Zurich
13683,Mondelēz International,United States of America,28720000000,2022,Chicago
13684,Danone,France,28708000000,2022,Paris


In [37]:
df_fortune.to_csv("Fortune 500.csv")
files.download("Fortune 500.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>